# Exercise 04

# Task 02: Extending the Evacuation Model: Introducing Facilitators #

ABM allows the exploration of different kinds of entities / agent classes. We can define various agent classes with specific objectives, actions, and properties. One way to extend the evacuation model is the introduction of facilitator agents. These are more experienced and know better about the exits, and also never panic. We will add facilitator agents and analyse their impact on evacuation time.

## 3.1 Implementation
1. Add the new Facilitator class to abmodel/agent.py
There are some comments as hints in the code where to place the new code. Also place the code junks (from agent.py / from model.py) here:

In [1]:
# place new code from agent.py here (for inspection)
import sys
sys.path.insert(0,'../../')
sys.path.insert(0,'../../abmodel')
from abmodel.fire_evacuation.agent import Human, FireExit, Wall, Sight     
       
# Add the new Facilitator class here!
class Facilitator(Human):
    def __init__(self,
            unique_id,
            speed: int,
            orientation: Human.Orientation.NORTH,
            nervousness: float,
            cooperativeness: float,
            believes_alarm: bool,
            model
            ):
        
        super().__init__(unique_id, speed,
            orientation,
            nervousness,
            cooperativeness,
            believes_alarm,
            model)
        

    def update_nervousness(self):
        # never get nervous
        crowdlevel = self.getCrowdLevel()
        if (crowdlevel > Human.CROWD_ANXIETY_THRESHOLD) and (self.nervousness + Human.CROWD_AXIETY_INCREASE < self.NERVOUSNESS_PANIC_THRESHOLD):
            # only increase nervousness if it doesn't reach panic level
            self.nervousness += Human.CROWD_AXIETY_INCREASE
        elif crowdlevel < Human.CROWD_RELAXATION_THRESHOLD:
            # decrease nervousness
            self.nervousness -= Human.CROWD_RELAXATION_DECREASE
        # valide numbers
        self.nervousness = min(max(0.0, self.nervousness), 1.0) 

In [2]:
# place new code from model.py here (for inspection)
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import time
import math

from mesa import Model
from mesa.datacollection import DataCollector
from mesa.space import Coordinate, MultiGrid
from mesa.time import RandomActivation

sys.path.insert(0,'../../')
from abmodel.fire_evacuation.agent import Human, Wall, FireExit, Facilitator


class FireEvacuation(Model):
    
    MIN_SPEED = 0
    MAX_SPEED = 3

    COOPERATE_WO_EXIT = False
    
    def __init__(
        self,
        floor_size: int,
        human_count: int,
        visualise_vision = True,
        random_spawn = True,
        alarm_believers_prop = 0.9,
        max_speed = 1,
        cooperation_mean = 0.3,
        nervousness_mean = 0.3,
        seed = 1,
        facilitators_percentage = 0.1,
     ):
        """
        

        Parameters
        ----------
        floor_size : int
            size of the room excluding walls.
        human_count : int
            DESCRIPTION.
        visualise_vision : bool
            DESCRIPTION.
        random_spawn : bool
            DESCRIPTION.
        save_plots : bool
            DESCRIPTION.
         : TYPE
            DESCRIPTION.

        Returns
        -------
        None.

        """
        super().__init__()
        self.facilitators_percentage = facilitators_percentage
        
        if human_count > floor_size ** 2:
            raise ValueError("Number of humans to high for the room!")
 
        
        # Not necessary?! 
        np.random.seed(seed)
        self.rng = np.random.default_rng(seed)
        self.MAX_SPEED = max_speed
        self.COOPERATE_WO_EXIT = FireEvacuation.COOPERATE_WO_EXIT
        
        self.stepcounter = -1
        
        # Create floorplan
        floorplan = np.full((floor_size + 2, floor_size + 2), '_')
        floorplan[(0,-1),:]='W'
        floorplan[:,(0,-1)]='W'
        floorplan[math.floor((floor_size + 2)/2),(0,-1)] = 'E'
        floorplan[(0,-1), math.floor((floor_size + 2)/2)] = 'E'

        # Rotate the floorplan so it's interpreted as seen in the text file
        floorplan = np.rot90(floorplan, 3)

        # Init params
        self.width = floor_size + 2
        self.height = floor_size + 2
        self.human_count = human_count
        self.visualise_vision = visualise_vision

        # Set up model objects
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(floor_size + 2, floor_size + 2, torus=False)

        # Used to easily see if a location is a FireExit, since this needs to be done a lot
        self.fire_exits: dict[Coordinate, FireExit] = {}

        # If random spawn is false, spawn_pos_list will contain the list of possible 
        # spawn points according to the floorplan
        self.random_spawn = random_spawn
        self.spawn_pos_list: list[Coordinate] = []

        self.decisioncount = dict()
        
        # Load floorplan objects
        for (x, y), value in np.ndenumerate(floorplan):
            pos: Coordinate = (x, y)

            value = str(value)
            floor_object = None
            if value == "W":
                floor_object = Wall(self)
            elif value == "E":
                floor_object = FireExit(self)
                self.fire_exits[pos] = floor_object
            elif value == "S":
                self.spawn_pos_list.append(pos)
            if floor_object:
                self.grid.place_agent(floor_object, pos)
                self.schedule.add(floor_object)

        # Create a graph of traversable routes, used by humans for pathing
        self.graph = nx.Graph()
        for agents, pos in self.grid.coord_iter():
            # If the location is empty, or there are no non-traversable humans
            if len(agents) == 0 or not any(not agent.traversable for agent in agents):
                neighbors_pos = self.grid.get_neighborhood(
                    pos, moore=True, include_center=True, radius=1
                )

                for neighbor_pos in neighbors_pos:
                    # If the neighbour position is empty, or no non-traversable 
                    # contents, add an edge
                    if self.grid.is_cell_empty(neighbor_pos) or not any(
                        not agent.traversable
                        for agent in self.grid.get_cell_list_contents(neighbor_pos)
                    ):
                        self.graph.add_edge(pos, neighbor_pos)

        # Collects statistics from our model run
        self.datacollector = DataCollector(
            {
                "NumEscaped" : lambda m: self.get_num_escaped(m),
                "AvgNervousness": lambda m: self.get_human_nervousness(m),
                "AvgSpeed": lambda m: self.get_human_speed(m),
             }
        )
        
        # Start placing human humans
        for i in range(0, self.human_count):
            if self.random_spawn:  # Place human humans randomly
                pos = tuple(self.rng.choice(tuple(self.grid.empties)))
            else:  # Place human humans at specified spawn locations
                pos = self.rng.choice(self.spawn_pos_list)

            if pos:
                # Create a random human
                speed = self.rng.integers(self.MIN_SPEED, self.MAX_SPEED + 1)

                nervousness = -1
                while nervousness < 0 or nervousness > 1:
                    nervousness = self.rng.normal(loc = nervousness_mean, scale = 0.2)
                    
                cooperativeness = -1
                while cooperativeness < 0 or cooperativeness > 1:
                    cooperativeness = self.rng.normal(cooperation_mean)

                belief_distribution = [alarm_believers_prop, 1 - alarm_believers_prop]
                believes_alarm = self.rng.choice([True, False], p=belief_distribution)

                orientation = Human.Orientation(self.rng.integers(1,5))
                
                # decide here whether to add a facilitator
                if np.random.rand() <= self.facilitators_percentage:
                    human = Facilitator(
                        i,
                        speed=speed,
                        orientation=orientation,
                        nervousness=nervousness*0.5,    # is always not-nervous
                        cooperativeness=cooperativeness,
                        believes_alarm=believes_alarm,
                        model=self
                    )
                else:
                    human = Human(
                        i,
                        speed=speed,
                        orientation=orientation,
                        nervousness=nervousness,
                        cooperativeness=cooperativeness,
                        believes_alarm=believes_alarm,
                        model=self,
                    )

                self.grid.place_agent(human, pos)
                self.schedule.add(human)
            else:
                print("No tile empty for human placement!")

        self.running = True


In [3]:
import sys
sys.path.insert(0,'../../abmodel')
from mesa.visualization import JupyterViz
from abmodel.fire_evacuation.model import FireEvacuation
from abmodel.fire_evacuation.agent import Human, FireExit, Wall, Sight

 3. Add a new model parameter facilitators_percentage as slider to model_params

In [4]:
model_params = {
    # "seed":  mesa.visualization.Number
    #      name="Random seed", value=1
    # ),
    "floor_size": {
        "type": "SliderInt",
        "value": 12,
        "label": "Room size (edge)",
        "min": 5,
        "max": 30,
        "step": 1,
    },
    "human_count": {
        "type": "SliderInt",
        "value": 80,
        "label": "Number Of Human Agents",
        "min": 1,
        "max": 500,
        "step": 5,
    },
    "max_speed": {
        "type": "SliderInt",
        "value": 2,
        "label": "Maximum Speed of agents",
        "min": 1,
        "max": 5,
        "step": 1,
    },
    "alarm_believers_prop": {
        "type": "SliderFloat",
        "value": 1.0,
        "label": "Proportion of Alarm Believers",
        "min": 0.0,
        "max": 1.0,
        "step": 0.05,
    },
    "cooperation_mean": {
        "type": "SliderFloat",
        "value": 0.3,
        "label": "Mean Cooperation",
        "min": 0.0,
        "max": 1.0,
        "step": 0.01,
    },
    "nervousness_mean": {
        "type": "SliderFloat",
        "value": 0.3,
        "label": "Mean Nervousness",
        "min": 0.0,
        "max": 1.0,
        "step": 0.01,
    },
        
    ## add slider for facilitators_percentage    
    "facilitators_percentage": {
        "type": "SliderInt",
        "value": 10,       # percentage in [0,1]
        "label": "facilitators percentage",
        "min": 0,
        "max": 100,
        "step": 1,   # percentage step 0.1
    },
}

 4. Add a new facilitator icon

In [5]:
def agent_portrayal(agent):
    size = 1
    nervousness = None
    
    if type(agent) is Human:
        nervousness = agent.nervousness
        if agent.nervousness > Human.NERVOUSNESS_PANIC_THRESHOLD:
            shape = "../../abmodel/fire_evacuation/resources/panicked_human.png"
        elif agent.humantohelp is not None:
            shape = "../../abmodel/fire_evacuation/resources/cooperating_human.png"
        else:
            shape = "../../abmodel/fire_evacuation/resources/human.png"
    if type(agent) is FireExit:
        shape = "../../abmodel/fire_evacuation/resources/fire_exit.png"
    if type(agent) is Wall:
        shape = "../../abmodel/fire_evacuation/resources/wall.png"
    if type(agent) is Sight:
        shape = "../../abmodel/fire_evacuation/resources/eye.png"
    if type(agent) is Facilitator:
        shape = "../../abmodel/fire_evacuation/resources/facilitator.png"   # for some reason this picture is coloured green
        
    return {"size": size,
            "shape": shape,
            "Nervousness": nervousness}

## 3.3 Model Run
Run the model in a console using instructions from the mesa tutorial.

In [6]:
page = JupyterViz(
    FireEvacuation,
    model_params,
    measures=["AvgNervousness"],
    name="Evacuation Model",
    agent_portrayal=agent_portrayal,
    space_drawer = "default",
)

page

Cannot show ipywidgets in text

# Observations
- How does the evacuation time change with the number of facilitators?
More facilitators lead to a faster evacuation time. 
- How does the graph change for different numbers of facilitators?
    - form of slope stays the same
    - facilitator percentage inversely proportional to maximal average nervousness
        -> logical, bc each facilitator 'adds' a zero to the average/ summed nervousness
        -> the more facilitators, the less overall (summed) nervousness -> smaller average nervousness
    - e.g. 10 % facilitators -> 0.175 max average nervousness
    - e.g. 80 % facilitators -> 0.025 max average nervousness
- change mean nervousness to 0.7
    - overall maximal average nervousness is higher
    - e.g. 10 % facilitators -> 0.42 max average nervousness
    - e.g. 80 % facilitators -> 0.07 max average nervousness

- sometimes agents just freeze until the end of simulation, without leaving the room